In [1]:
#import libraries
import boto3
import json
import time

In [2]:
# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_JordiPlanas_Made_in_game.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
s3BucketName = "wri-testing"
region = 'eu-central-1'

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [3]:
def startJob(s3BucketName, objectName):
    response = None
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.start_document_text_detection(DocumentLocation={
        'S3Object': {'Bucket': s3BucketName,
                     'Name': objectName
                    }
        })
    return response["JobId"]
def isJobComplete(jobId):
 # For production use cases, use SNS based notification 
 # Details at: https://docs.aws.amazon.com/textract/latest/dg/api-async.html
    time.sleep(5)
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))
    return status
def getJobResults(jobId):
    pages = []
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
        while(nextToken):
            response = client.get_document_text_detection(JobId=jobId, NextToken=nextToken)
            pages.append(response)
            print("Resultset page recieved: {}".format(len(pages)))
            nextToken = None
            if('NextToken' in response):
                nextToken = response['NextToken']
    return pages

def save_txt_file(file, file_name):
    f = open(file_name, "w")
    f.write(file)
    f.close()

In [9]:
def AWS_pdf_to_text(key, s3BucketName):
    # Document
    documentName = key
    jobId = startJob(s3BucketName, documentName)
    print("Started job with id: {}".format(jobId))
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
    #print(response)
    # Print detected text
    policy = ""
    for resultPage in response:
        print(resultPage)
        for item in resultPage["Blocks"]:
            if item["BlockType"] == "LINE":
                policy = policy + item["Text"] + "\n"
                print ('\033[94m' + item["Text"] + '\033[0m')
    return policy

In [12]:
i = 0
keyword = "095157a12eca16947dd937dbd56ed8939d2967d7"
lower_limit = 0
upper_limit = lower_limit + 2
for obj in s3.Bucket(s3BucketName).objects.all():#.filter(Prefix='El Salvador'):
    if keyword in obj.key: #lower_limit < i < upper_limit:
        print(i, "**", obj.key)
        policy = AWS_pdf_to_text(obj.key, s3BucketName)
        save_txt_file(policy, "/home/propietari/Documents/S3/" + obj.key[:-4] + ".txt")
    i += 1

17 ** dof/095157a12eca16947dd937dbd56ed8939d2967d7.txt
Started job with id: 3b225aacabab0f719362b4b0b4e26773068327361d4b096805cb6b81abb786af
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: FAILED
Resultset page recieved: 1
{'JobStatus': 'FAILED', 'StatusMessage': 'INVALID_IMAGE_TYPE', 'DetectDocumentTextModelVersion': '1.0', 'ResponseMetadata': {'RequestId': '7720a0b5-029d-4ae7-b911-40d7c8f5fc60', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '7720a0b5-029d-4ae7-b911-40d7c8f5fc60', 'content-type': 'application/x-amz-json-1.1', 'content-length': '98', 'date': 'Sat, 06 Mar 2021 09:29:57 GMT'}, 'RetryAttempts': 0}}


In [ ]:
policy

In [13]:
path = "/home/propietari/Documents/S3/"
file_name = "095157a12eca16947dd937dbd56ed8939d2967d7.txt"
file = path + file_name
f = open(file, "r")
print(f.read())


<html>
<head>
<title>Federal Register, Volume 86 Issue 42 (Friday, March 5, 2021)</title>
</head>
<body><pre>
[Federal Register Volume 86, Number 42 (Friday, March 5, 2021)]
[Notices]
[Pages 12942-12943]
From the Federal Register Online via the Government Publishing Office [<a href="http://www.gpo.gov">www.gpo.gov</a>]
[FR Doc No: 2021-04543]


-----------------------------------------------------------------------

ENVIRONMENTAL PROTECTION AGENCY

[ER-FRL-9055-5]


Environmental Impact Statements; Notice of Availability

    Responsible Agency: Office of Federal Activities, General 
Information 202-564-5632 or <a href="https://www.epa.gov/nepa">https://www.epa.gov/nepa</a>.


[[Page 12943]]


Weekly receipt of Environmental Impact Statements (EIS)
Filed February 22, 2021 10 a.m. EST Through March 1, 2021 10 a.m. EST
Pursuant to 40 CFR 1506.9.

    Notice: Section 309(a) of the Clean Air Act requires that EPA make 
public its comments on EISs issued by other Federal agencies. EPA's 
co